In [1]:
import json
from time import sleep
import pickle
import re
import collections
import pandas as pd
pd.set_option('display.max_rows', 100)
from os import listdir
from os.path import isfile, join
import numpy as np
import sys
import math

# find representatives

In [2]:
rep_path = 'C:\\Users\\chenwang\\Documents\\courses\\FA16\\CS597\\twitterMining\\InitialSet\\Football\\location\\rep\\'
path = 'C:\\Users\\chenwang\\Documents\\courses\\FA16\CS597\\twitterMining\\InitialSet\\Football\\location\\'
with open (rep_path + 'followees_list.pickle','rb+') as f:
    followees_list = pickle.load(f)

In [3]:
len(followees_list)

83

In [4]:
def find_common_followee(followees_list):
    
    followee_vocab = {}

    for user in followees_list.keys():
        counter = collections.Counter(followees_list[user])
        for c in counter.keys():
            if c not in followee_vocab.keys():
                followee_vocab[c] = counter[c]
            else:
                followee_vocab[c] += counter[c]
    
    ranked_followees = sorted(followee_vocab.items(),key=lambda x:x[1],reverse=True)[:300]
    
    return ranked_followees

# most common followees in the inital set (raw)

In [5]:
with open(path + 'de_common_followees.pickle','rb+') as f:
    details = pickle.load(f)

In [7]:
df_raw = pd.DataFrame(details, columns = ['id','screen_name','location','counts','total_followers'])
df_raw.head()

,id,screen_name,location,counts,total_followers
0,51263592,AdamSchefter,New York,34,5448922
1,26257166,SportsCenter,"Bristol, CT",31,30150419
2,19426551,NFL,,29,20381637
3,16212685,chicagobulls,Chicago,26,2950194
4,2557521,espn,"Bristol, CT",26,29848501


# leverage 

too many features (<u>92 features</u>)

<b>combine Amin's approach rank these</b>

Rank: (count of following in the initial list)/(count of followers of that rep node)

In [8]:
df_raw['IDF'] = df_raw['counts']/df_raw['total_followers']
df_ranked = df_raw.sort_values(by='IDF',ascending =False)
df_ranked.head(100)

,id,screen_name,location,counts,total_followers,IDF
110,635721381,AaronLemingNFL,Southern California,9,8897,0.001012
288,2162089176,Dylan120Sports,Chicago,6,6009,0.000999
175,222249852,CEmma670,Chicago,8,8763,0.000913
223,611931261,CSNHayes,Chicago,7,14367,0.000487
176,85675782,ByScottPowers,"Chicago, IL",8,16814,0.000476
173,164315100,djdurkin,"Chicago, IL",8,18265,0.000438
145,46933773,ChuckGarfien,Chicago,8,21506,0.000372
201,18906156,Rich_Campbell,Chicago,7,22232,0.000315
142,121509578,AdamHoge,Snapchat: AdamAHoge,8,25600,0.000313
196,61658283,scottmerkin,Chicago,7,23350,0.000300


# find prior
<img src = 'prior.png'>

In [9]:
df_ranked['Tmax'] = df_ranked['total_followers']/(df_ranked['counts']/len(followees_list))

# probability of user in the inital set following this rep
#df_ranked['I_Vx=1'] = df_ranked['counts']/len(followees_list)
# probability of general user set following this rep
# assuming twitter has a total of 883000000 users
#df_ranked['Vx=1'] = df_ranked['total_followers']/223000000

#p(follow|T)/P(follow)
df_ranked['follow'] = df_ranked['counts']/df_ranked['total_followers']
#p(not_follow|T)/p(not_follow)
df_ranked['unfollow'] = (len(followees_list)-df_ranked['counts'])/(223000000-df_ranked['total_followers'])

df_ranked.head(30)

,id,screen_name,location,counts,total_followers,IDF,Tmax,follow,unfollow
110,635721381,AaronLemingNFL,Southern California,9,8897,0.001012,82050.111111,0.001012,3.318518e-07
288,2162089176,Dylan120Sports,Chicago,6,6009,0.000999,83124.500000,0.000999,3.453008e-07
175,222249852,CEmma670,Chicago,8,8763,0.000913,90916.125000,0.000913,3.363361e-07
223,611931261,CSNHayes,Chicago,7,14367,0.000487,170351.571429,0.000487,3.408291e-07
176,85675782,ByScottPowers,"Chicago, IL",8,16814,0.000476,174445.250000,0.000476,3.363482e-07
173,164315100,djdurkin,"Chicago, IL",8,18265,0.000438,189499.375000,0.000438,3.363504e-07
145,46933773,ChuckGarfien,Chicago,8,21506,0.000372,223124.750000,0.000372,3.363553e-07
201,18906156,Rich_Campbell,Chicago,7,22232,0.000315,263608.000000,0.000315,3.408412e-07
142,121509578,AdamHoge,Snapchat: AdamAHoge,8,25600,0.000313,265600.000000,0.000313,3.363615e-07
196,61658283,scottmerkin,Chicago,7,23350,0.000300,276864.285714,0.000300,3.408429e-07


In [11]:
weight = df_ranked['IDF'].tolist()[0:100]
with open(path + 'weight_rep.pickle','wb+') as f:
    pickle.dump(weight, f)